# <font color=green>*BCBIO 490*</font>

In [75]:
import numpy as np
import os
import math
from Bio import SeqIO
import blosum as bl

# The class that performs sequence alignments on two given sequences based on user-defined scores and penalties.
class Sequence_Alignment:
    
    # Constructor method 
    # @param match : score used if two characters are identical
    # @param mismatch : score used if two characters are not identical
    # @param gapOpen : penalty for creating a gap '-' in an alignment
    # @param gapExtend : penalty for creating a consecutive gap 
    # @param diff_block : penalty for a block consisting of alignments in which large regions of both sequences  
    #                     are different. 
    # @param ambiguous : score used if one of the two characters is 'N', which is a sequencing error in which 
    #                    the particular nucleotide is unknown. 
    def __init__(self, match=10, mismatch=-20, gapOpen=40, gapExtend=2, diff_block=300, ambiguous=0):
        self.gapOpen = abs(gapOpen)
        self.gapExtend = abs(gapExtend)
        self.diff = abs(diff_block)
        self.sub_mat = np.zeros((129,129))
        for i in range(len(self.sub_mat)):
            for j in range(len(self.sub_mat[i])):
                if i == 'N' or j == 'N':
                    self.sub_mat[i][j] = 0 
                elif i != j:
                    self.sub_mat[i][j] = mismatch 
                else:
                    self.sub_mat[i][j] = match
                    
        self.protein = bl.BLOSUM(62) # Scoring system for protein alignments
    
    # Used for accessing the scoring matrix generated in the constructor method given two characters
    # @param char1 : first character
    # @param char2 : second character
    # @param mode : determines the scoring system that the algorithm will use. Default is 0, which is the 
    #               scoring system for DNA/ RNA sequences. If not 0, the algorithm uses the scoring system 
    #               for protein sequences. 
    def getScore(self, char1, char2, mode=0):
        return self.sub_mat[ord(char1.upper())][ord(char2.upper())] if mode == 0 \
                else self.protein[char1.upper()][char2.upper()]
    
    # Modifies the gapOpen variable 
    def setGapOpenPenalty(self, penalty):
        self.gapOpen = abs(penalty)
    
    # Modifies the gapExtend variable
    def setGapExtensionPenalty(self, penalty):
        self.gapExtend = abs(penalty)
    
    # Modifies the diff variable
    def setDifferenceBlockPenalty(self, penalty):
        self.diff = abs(penalty)
    
    # Method that takes in two FASTA files containing same type of sequences, and aligns them based on the 
    # alignment type specified by the user. 
    # @param fasta1 : file name of the first FASTA file 
    # @param fasta2 : file name of the second FASTA file
    # @align_type : determines the alignment type. Possible values are 'global', 'local', or 'gap3'
    def alignInput(self, fasta1, fasta2, align_type):
        """
        cwd = os.getcwd()  # Get the current working directory (cwd)
        files = os.listdir(cwd)  # Get all the files in that directory
        print("Files in %r: %s" % (cwd, files))
        """
        if os.path.getsize(fasta1) == 0 or os.path.getsize(fasta2) == 0:
            return "Input contains empty file(s)"
        if align_type != 'global' and align_type != 'local' and align_type != 'gap3':
            return "This program only has three alignment types: 'global' or 'local' or 'gap3'"
                    
        with open(fasta1, "r") as handle1, open(fasta2, "r") as handle2:
            for record1, record2 in zip(SeqIO.parse(handle1, "fasta"), SeqIO.parse(handle2, "fasta")):
                print("Aligning " + record1.id + " with " + record2.id + " using " + align_type + " alignment: ")
                print()
                if align_type == 'global':
                    self.global_alignment(record1.seq, record2.seq)
                elif align_type == 'local':
                    self.local_alignment(record1.seq, record2.seq)
                elif align_type == 'gap3':
                    self.gap3(record1.seq, record2.seq)

    # Method that allows user to manually create two sequences and align them based on the alignment type
    # specified by the user.
    # @param seq_type : determines the type of sequences that can be generated. Possible values are 
    #                   'DNA', 'RNA', or 'PROTEIN'
    def manualAlign(self, seq_type):
        seq = {'DNA': ('ATCG', 0), 'RNA': ('AUCG', 0), 'PROTEIN': ('AGVLISTCMPFYWDENQKRH', 1)}
        if seq_type.upper() not in seq:
            print("Choose either 'DNA', 'RNA', or 'PROTEIN' sequences. No other types of sequences are allowed.")
            return 

        while True:
            seq1 = input("Enter a " + seq_type.upper() + " sequence to be aligned: ")
            validation = [i.upper() in seq[seq_type.upper()][0] for i in seq1]
            if (all(validation)):
                break
            print("***" + seq_type.upper() + "sequences should contain only these characters: " + \
                  seq[seq_type.upper()][0] + ", and no spaces are allowed ! ***")
        
        print("------------------------------------------------------------------------------------------------")
        
        while True:
            seq2 = input("Enter another " + seq_type.upper() + " sequence to be aligned: ")
            validation = [i.upper() in seq[seq_type.upper()][0] for i in seq2]
            if (all(validation)):
                break
            print("***" + seq_type.upper() + "sequences should contain only these characters: " + \
                  seq[seq_type.upper()][0] + ", and no spaces are allowed ! ***")
            print()
        
        print("------------------------------------------------------------------------------------------------")
        
        print("Please choose an alignment mode: ")
        print("\t" + "1. Type 'global' for global alignment")
        print("\t" + "2. Type 'local' for local alignment")
        print("\t" + "3. Type 'gap3' for gap3 alignment")
        print()
        
        while True:
            align = input("Enter your desired alignment type: ")
            if align.lower() == 'global' or align.lower() == 'local' or align.lower() == 'gap3':
                break
            print("***Please select alignment type only from 'global' or 'local or 'gap3' ! ***")
            print()
            
        print("------------------------------------------------------------------------------------------------")
        print()
        print("Outputting " + align.lower() + " Alignment ...")
        print()
        if align.lower() == 'local':
            self.local_alignment(seq1, seq2, seq[seq_type.upper()][1])
        elif align.lower() == 'global':
            self.global_alignment(seq1, seq2, seq[seq_type.upper()][1])
        else:
            self.gap3(seq1, seq2, seq[seq_type.upper()][1])
    
    # Traceback method for generating the global alignment of two given sequences, which is only called by
    # the method global_alignment_trace when the first sequence has a size below a certain threshold. 
    # @param seq1 : first sequence
    # @param seq2 : second sequence
    # @param S : the score used as the basis of the construction of s_mat
    # @param D : the score used as the basis of the construction of d_mat
    # @param curr : the matrix to start from when generating the optimal alignment
    # @param mode : determines the scoring system that the algorithm will use. 0 is the 
    #               scoring system for DNA/ RNA sequences. If not 0, the algorithm uses the scoring system 
    #               for protein sequences. 
    def global_traceback(self, seq1, seq2, S, D, curr, mode):
        M = len(seq1)
        N = len(seq2)
        
        seq1 = " " + seq1
        seq2 = " " + seq2
        
        s_mat = np.zeros((M + 1, N + 1))
        i_mat = np.zeros((M + 1, N + 1))      # Generation of score matrices using suffixes of both sequences
        d_mat = np.zeros((M + 1, N + 1))
        
        s_mat[M][N] = S
        d_mat[M][N] = D
        i_mat[M][N] = s_mat[M][N] - self.gapOpen
        
        for i in range(N - 1, -1, -1):
            i_mat[M][i] = i_mat[M][i + 1] - self.gapExtend
            s_mat[M][i] = i_mat[M][i]
            d_mat[M][i] = s_mat[M][i] - self.gapOpen
        
        for j in range(M - 1, -1, -1):
            d_mat[j][N] = d_mat[j + 1][N] - self.gapExtend
            s_mat[j][N] = d_mat[j][N]
            i_mat[j][N] = s_mat[j][N] - self.gapOpen
            
            for k in range(N - 1, -1, -1):
                d_mat[j][k] = max(d_mat[j + 1][k] - self.gapExtend, s_mat[j + 1][k] - self.gapOpen - self.gapExtend)
                i_mat[j][k] = max(i_mat[j][k + 1] - self.gapExtend, s_mat[j][k + 1] - self.gapOpen - self.gapExtend)
                s_mat[j][k] = max(s_mat[j + 1][k + 1] + self.getScore(seq1[j + 1], seq2[k + 1], mode), d_mat[j][k], i_mat[j][k])
                
        current = curr
        i = j = 0
        align_seq1 = ""
        align_seq2 = ""
        align_mid = ""

        while i <= M and j <= N:
            if current == 'S':
                if i == M and j == N:
                    break
                elif i == M or s_mat[i][j] == i_mat[i][j]:
                    current = 'I'
                    continue
                elif j == N or s_mat[i][j] == d_mat[i][j]:
                    current = 'D'
                    continue
                    
                align_seq1 += seq1[i + 1]
                align_seq2 += seq2[j + 1]
                if seq1[i + 1].upper() != seq2[j + 1].upper():
                    if self.getScore(seq1[i + 1].upper(), seq2[j + 1].upper(), mode) > 0:
                        align_mid += "="
                    else:
                        align_mid += "*"
                else:
                    align_mid += "|"
                i += 1
                j += 1
                continue
                
            elif current == 'I':
                align_seq1 += '-'
                align_seq2 += seq2[j + 1]
                align_mid += " "
                if (j == N - 1) or (i_mat[i][j] == s_mat[i][j + 1] - self.gapOpen - self.gapExtend):
                    current = 'S'
                j += 1
                continue
                
            elif current == 'D':
                align_seq1 += seq1[i + 1]
                align_seq2 += '-'
                align_mid += " "
                if (i == M - 1) or (d_mat[i][j] == s_mat[i + 1][j] - self.gapOpen - self.gapExtend):
                    current = 'S'
                i += 1
                continue
        
        return [s_mat[0][0], align_seq1, align_mid, align_seq2]
    
    # Divide-and-Conquer method for generating the global alignment of two given sequences, which allows for
    # the analysis of large datasets. 
    # @param seq1 : first sequence
    # @param seq2 : second sequence
    # @param s : the score used as the basis of the construction of s_mat
    # @param d : the score used as the basis of the construction of d_mat
    # @param S : the score used as the basis of the construction of S_MAT
    # @param D : the score used as the basis of the construction of D_MAT
    # @param curr : the matrix to start from when generating the optimal alignment
    # @param mode : determines the scoring system that the algorithm will use. 0 is the 
    #               scoring system for DNA/ RNA sequences. If not 0, the algorithm uses the scoring system 
    #               for protein sequences. 
    def global_alignment_trace(self, seq1, seq2, s, d, S, D, curr, mode):
        
        # When the first sequence is less than 50 characters, call global_traceback method
        if len(seq1) <= 50: 
            arr = self.global_traceback(seq1, seq2, S, D, curr, mode)
            return arr
        
        else:
            M = len(seq1)
            N = len(seq2)
            
            # Divide the first sequence into half
            imid = M // 2
        
            seq1 = " " + seq1
            seq2 = " " + seq2
            s_row = np.zeros(N + 1)
            i_row = np.zeros(N + 1)           # Generation of score array using prefixes of both sequences
            d_row = np.zeros(N + 1)
        
            S_ROW = np.zeros(N + 1)
            I_ROW = np.zeros(N + 1)           # Generation of score array using suffixes of both sequences
            D_ROW = np.zeros(N + 1)
        
            s_row[0] = s
            i_row[0] = s_row[0] - self.gapOpen
            d_row[0] = d
        
            S_ROW[N] = S
            I_ROW[N] = S_ROW[N] - self.gapOpen
            D_ROW[N] = D
            
            for j in range(1, N + 1):
                i_row[j] = i_row[j-1] - self.gapExtend
                s_row[j] = i_row[j]
                d_row[j] = s_row[j] - self.gapOpen
            
            for i in range(1, imid + 1):
                s_prev = s_row[0]
            
                d_row[0] = d_row[0] - self.gapExtend
                s_row[0] = d_row[0]
                i_row[0] = s_row[0] - self.gapOpen
            
                for j in range(1, N + 1):
                    d_row[j] = max(d_row[j] - self.gapExtend, s_row[j] - self.gapOpen - self.gapExtend)
                    i_row[j] = max(i_row[j - 1] - self.gapExtend, s_row[j - 1] - self.gapOpen - self.gapExtend)
                    tmp = s_row[j]
                    s_row[j] = max(s_prev + self.getScore(seq1[i], seq2[j], mode), d_row[j], i_row[j])
                    s_prev = tmp

            for j in range(N - 1, -1, -1):
                I_ROW[j] = I_ROW[j + 1] - self.gapExtend
                S_ROW[j] = I_ROW[j]
                D_ROW[j] = S_ROW[j] - self.gapOpen
            
            for i in range(M - 1, imid - 1, -1):
                S_PREV = S_ROW[N]
            
                D_ROW[N] = D_ROW[N] - self.gapExtend
                S_ROW[N] = D_ROW[N]
                I_ROW[N] = S_ROW[N] - self.gapOpen
            
                for j in range(N - 1, -1, -1):
                    D_ROW[j] = max(D_ROW[j] - self.gapExtend, S_ROW[j] - self.gapOpen - self.gapExtend)
                    I_ROW[j] = max(I_ROW[j + 1] - self.gapExtend, S_ROW[j + 1] - self.gapOpen - self.gapExtend)
                    tmp = S_ROW[j]
                    S_ROW[j] = max(S_PREV + self.getScore(seq1[i + 1], seq2[j + 1], mode), D_ROW[j], I_ROW[j])
                    S_PREV = tmp
                    
            df = -math.inf
            st = -math.inf
        
            jd = 0
            js = 0
            
            # Find the point of intersection in which optimal alignment crosses to locate the partition point
            # of the second sequence. 
            for j in range(N + 1):
                if d_row[j] + D_ROW[j] + self.gapOpen > df:
                    df = d_row[j] + D_ROW[j] + self.gapOpen
                    jd = j
                if s_row[j] + S_ROW[j] > st:
                    st = s_row[j] + S_ROW[j]
                    js = j

            if max(df, st) == st:
                jmid = js
                final_score = st
                current = 'S'
                
            else:
                jmid = jd 
                final_score = df
                current = 'D'
            
            # Recursively call itself on the partitions of both sequences. 
            top_align = self.global_alignment_trace(seq1[1: imid + 1], seq2[1:jmid + 1], s, d, 
                                         S_ROW[jmid], D_ROW[jmid], curr, mode)
        
            bottom_align = self.global_alignment_trace(seq1[imid+1::], seq2[jmid+1::], s_row[jmid], 
                                            d_row[jmid], S, D, current, mode)
            
            # Merge the two generated optimal alignments together to solve the optimal alignment of the 
            # whole sequences.
            final_align_seq1 = top_align[1] + bottom_align[1]
            final_align_mid = top_align[2] + bottom_align[2]
            final_align_seq2 = top_align[3] + bottom_align[3]
        
            return [final_score, final_align_seq1, final_align_mid, final_align_seq2]
        
    # Method for global alignment, which aligns both given sequences end to end, and outputs the whole 
    # alignment. 
    # @param seq1 : first sequence
    # @param seq2 : second sequence 
    # @param mode : determines the scoring system that the algorithm will use. Default is 0, which is the 
    #               scoring system for DNA/ RNA sequences. If not 0, the algorithm uses the scoring system 
    #               for protein sequences. 
    def global_alignment(self, seq1, seq2, mode=0):
        s = 0
        d = -self.gapOpen
        if len(seq1) == 0 or len(seq2) == 0:
            print("One of the sequences is empty.")
            return
        
        arr = self.global_alignment_trace(seq1, seq2, s, d, s, d, 'S', mode)
        self.printAlignment(seq1, seq2, 0, 0, len(seq1), len(seq2), arr[0], arr[1], arr[2], arr[3])

    # Method for local alignment, which aligns one sequence to the other at regions where there is high
    # similarity, and outputs the region aligned. 
    # @param seq1 : first sequence
    # @param seq2 : second sequence 
    # @param mode : determines the scoring system that the algorithm will use. Default is 0, which is the 
    #               scoring system for DNA/ RNA sequences. If not 0, the algorithm uses the scoring system 
    #               for protein sequences. 
    def local_alignment(self, seq1, seq2, mode=0):
        if len(seq1) == 0 or len(seq2) == 0:
            return
        
        highestScore = 0
        firstRow = len(seq1)
        firstCol = len(seq2)
        
        s_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        i_mat = np.zeros((len(seq1) + 1, len(seq2) + 1)) # Generation of score array using suffixes of both sequences
        d_mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
        
        d_mat[len(seq1)][len(seq2)] = -(self.gapOpen + self.gapExtend)
        i_mat[len(seq1)][len(seq2)] = -(self.gapOpen + self.gapExtend)
        for i in range(len(seq2) - 1, -1, -1):
            i_mat[len(seq1)][i] =  -(self.gapOpen + self.gapExtend)
            d_mat[len(seq1)][i] =  -(self.gapOpen + self.gapExtend)
        
        for j in range(len(seq1) - 1, -1, -1):
            d_mat[j][len(seq2)] =  -(self.gapOpen + self.gapExtend)
            i_mat[j][len(seq2)] =  -(self.gapOpen + self.gapExtend)
        
            for k in range(len(seq2) - 1, -1, -1):
                d_mat[j][k] = max(d_mat[j + 1][k] - self.gapExtend, s_mat[j + 1][k] - self.gapOpen - self.gapExtend)
                i_mat[j][k] = max(i_mat[j][k + 1] - self.gapExtend, s_mat[j][k + 1] - self.gapOpen - self.gapExtend)
                s_mat[j][k] = max(0, s_mat[j + 1][k + 1] + self.getScore(seq1[j], seq2[k], mode), d_mat[j][k], i_mat[j][k])
                
                if highestScore < s_mat[j][k]:
                    highestScore = s_mat[j][k]
                    firstRow = j
                    firstCol = k
                    
        current = 'S'
        i = firstRow
        j = firstCol 
        align_seq1 = ""
        align_seq2 = ""
        align_mid = ""

        while i <= len(seq1) and j <= len(seq2):
            if current == 'S':
                if i == len(seq1) or j == len(seq2) or s_mat[i][j] == 0:
                    break
                elif s_mat[i][j] == i_mat[i][j]:
                    current = 'I'
                    continue
                elif s_mat[i][j] == d_mat[i][j]:
                    current = 'D'
                    continue
                align_seq1 += seq1[i]
                align_seq2 += seq2[j]
                if seq1[i].upper() != seq2[j].upper():
                    if self.getScore(seq1[i].upper(), seq2[j].upper(), mode) > 0:
                        align_mid += "="
                    else:
                        align_mid += "*"
                else:
                    align_mid += "|"
                i += 1
                j += 1
                continue
                
            elif current == 'I':
                align_seq1 += '-'
                align_seq2 += seq2[j]
                align_mid += " "
                if (j == len(seq2) - 1) or (i_mat[i][j] == s_mat[i][j + 1] - self.gapOpen - self.gapExtend):
                    current = 'S'
                j += 1
                continue
                
            elif current == 'D':
                align_seq1 += seq1[i]
                align_seq2 += '-'
                align_mid += " "
                if (i == len(seq1) - 1) or (d_mat[i][j] == s_mat[i + 1][j] - self.gapOpen - self.gapExtend):
                    current = 'S'
                i += 1
                continue 
                
        lastRow = i
        lastCol = j
        score = s_mat[firstRow][firstCol]
        
        self.printAlignment(seq1, seq2, firstRow, firstCol, lastRow, lastCol, score, align_seq1, align_mid, align_seq2)
    
    # Traceback method for generating the gap3 alignment of two given sequences, which is only called by
    # the method gap3_trace when the first sequence has a size below a certain threshold. 
    # @param seq1 : first sequence
    # @param seq2 : second sequence
    # @param S : the score used as the basis of the construction of s_mat
    # @param D : the score used as the basis of the construction of d_mat
    # @param H : the score used as the basis of the construction of h_mat
    # @param curr : the matrix to start from when generating the optimal alignment
    # @param mode : determines the scoring system that the algorithm will use. 0 is the 
    #               scoring system for DNA/ RNA sequences. If not 0, the algorithm uses the scoring system 
    #               for protein sequences. 
    def gap3_traceback(self, seq1, seq2, S, D, H, curr, mode):
        M = len(seq1)
        N = len(seq2)
        
        seq1 = " " + seq1
        seq2 = " " + seq2
        
        s_mat = np.zeros((M + 1, N + 1))
        i_mat = np.zeros((M + 1, N + 1))  # Generation of score array using suffixes of both sequences
        d_mat = np.zeros((M + 1, N + 1))
        h_mat = np.zeros((M + 1, N + 1))
        
        s_mat[M][N] = S 
        d_mat[M][N] = D 
        i_mat[M][N] = s_mat[M][N] - self.gapOpen
        h_mat[M][N] = H
        
        for i in range(N - 1, -1, -1):
            i_mat[M][i] = i_mat[M][i + 1] - self.gapExtend
            h_mat[M][i] = H 
            s_mat[M][i] = max(i_mat[M][i], h_mat[M][i])
            d_mat[M][i] = s_mat[M][i] - self.gapOpen
        
        for j in range(M - 1, -1, -1):
            d_mat[j][N] = d_mat[j + 1][N] - self.gapExtend
            h_mat[j][N] = H 
            s_mat[j][N] = max(d_mat[j][N], h_mat[j][N])
            i_mat[j][N] = s_mat[j][N] - self.gapOpen
            
            for k in range(N - 1, -1, -1):
                d_mat[j][k] = max(d_mat[j + 1][k] - self.gapExtend, s_mat[j + 1][k] - self.gapOpen - self.gapExtend)
                i_mat[j][k] = max(i_mat[j][k + 1] - self.gapExtend, s_mat[j][k + 1] - self.gapOpen - self.gapExtend)
                h_mat[j][k] = max(h_mat[j + 1][k], h_mat[j][k + 1], s_mat[j + 1][k] - self.diff, s_mat[j][k + 1] - self.diff)
                s_mat[j][k] = max(s_mat[j + 1][k + 1] + self.getScore(seq1[j + 1], seq2[k + 1], mode), d_mat[j][k], i_mat[j][k], h_mat[j][k])
                
        current = curr
        i = j = 0
        align_seq1 = ""
        align_seq2 = ""
        align_mid = ""
        
        while i <= M and j <= N:
            if current == 'S':
                if i == M and j == N:
                    break
                elif s_mat[i][j] == h_mat[i][j]:
                    current = 'H'
                    continue
                elif i == M or s_mat[i][j] == i_mat[i][j]:
                    current = 'I'
                    continue
                elif j == N or s_mat[i][j] == d_mat[i][j]:
                    current = 'D'
                    continue
                    
                align_seq1 += seq1[i + 1]
                align_seq2 += seq2[j + 1]
                if seq1[i + 1].upper() != seq2[j + 1].upper():
                    if self.getScore(seq1[i + 1].upper(), seq2[j + 1].upper(), mode) > 0:
                        align_mid += "="
                    else:
                        align_mid += "*"
                else:
                    align_mid += "|"
                i += 1
                j += 1
                continue
                
            elif current == 'I':
                align_seq1 += '-'
                align_seq2 += seq2[j + 1]
                align_mid += " "
                if (j == N - 1) or (i_mat[i][j] == s_mat[i][j + 1] - self.gapOpen - self.gapExtend):
                    current = 'S'
                j += 1
                continue
                
            elif current == 'D':
                align_seq1 += seq1[i + 1]
                align_seq2 += '-'
                align_mid += " "
                if (i == M - 1) or (d_mat[i][j] == s_mat[i + 1][j] - self.gapOpen - self.gapExtend):
                    current = 'S'
                i += 1
                continue
                
            elif current == 'H':
                if i == M and j == N:
                    break
                if i + 1 <= M:
                    if h_mat[i][j] == h_mat[i + 1][j] or h_mat[i][j] == s_mat[i + 1][j] - self.diff:
                        align_seq1 += seq1[i + 1]
                        align_seq2 += " "
                        align_mid += "+"
                        if h_mat[i][j] == s_mat[i + 1][j] - self.diff:
                            current = 'S'
                        i += 1
                        continue
                        
                if j + 1 <= N:
                    if h_mat[i][j] == h_mat[i][j + 1] or h_mat[i][j] == s_mat[i][j + 1] - self.diff:
                        align_seq1 += " "
                        align_seq2 += seq2[j + 1]
                        align_mid += "+"
                        if h_mat[i][j] == s_mat[i][j + 1] - self.diff:
                            current = 'S'
                        j += 1
                        continue

        return [s_mat[0][0], align_seq1, align_mid, align_seq2]
    
    # Divide-and-Conquer method for generating the gap3 alignment of two given sequences, which allows for
    # the analysis of large datasets. 
    # @param seq1 : first sequence
    # @param seq2 : second sequence
    # @param s : the score used as the basis of the construction of s_mat
    # @param d : the score used as the basis of the construction of d_mat
    # @param h : the score used as the basis of the construction of h_mat
    # @param S : the score used as the basis of the construction of S_MAT
    # @param D : the score used as the basis of the construction of D_MAT
    # @param H : the score used as the basis of the construction of H_MAT
    # @param curr : the matrix to start from when generating the optimal alignment
    # @param mode : determines the scoring system that the algorithm will use. 0 is the 
    #               scoring system for DNA/ RNA sequences. If not 0, the algorithm uses the scoring system 
    #               for protein sequences. 
    def gap3_trace(self, seq1, seq2, s, d, h, S, D, H, curr, mode):
        
        # When the first sequence is less than 50 characters, call gap3_traceback method
        if len(seq1) <= 50:
            arr = self.gap3_traceback(seq1, seq2, S, D, H, curr, mode)
            return arr
        else: 
            M = len(seq1)
            N = len(seq2)
            imid = M // 2
            
            seq1 = " " + seq1
            seq2 = " " + seq2
            s_row = np.zeros(N + 1)
            i_row = np.zeros(N + 1)   # Generation of score array using prefixes of both sequences
            d_row = np.zeros(N + 1)   
            h_row = np.zeros(N + 1)
        
            S_ROW = np.zeros(N + 1)
            I_ROW = np.zeros(N + 1)   # Generation of score array using suffixes of both sequences
            D_ROW = np.zeros(N + 1)
            H_ROW = np.zeros(N + 1)
        
            s_row[0] = s
            i_row[0] = s_row[0] - self.gapOpen
            d_row[0] = d
            h_row[0] = h
        
            S_ROW[N] = S
            I_ROW[N] = S_ROW[N] - self.gapOpen
            H_ROW[N] = H
            D_ROW[N] = D
        
            for j in range(1, N + 1):
                i_row[j] = i_row[j-1] - self.gapExtend
                h_row[j] = h
                s_row[j] = max(i_row[j], h_row[j])
                d_row[j] = s_row[j] - self.gapOpen

            for i in range(1, imid + 1):
                s_prev = s_row[0]
            
                d_row[0] = d_row[0] - self.gapExtend
                h_row[0] = h
                s_row[0] = max(d_row[0], h_row[0])
                i_row[0] = s_row[0] - self.gapOpen
            
                for j in range(1, N + 1):
                    d_row[j] = max(d_row[j] - self.gapExtend, s_row[j] - self.gapOpen - self.gapExtend)
                    i_row[j] = max(i_row[j - 1] - self.gapExtend, s_row[j - 1] - self.gapOpen - self.gapExtend)
                    h_row[j] = max(h_row[j - 1], h_row[j], s_row[j - 1] - self.diff, s_row[j] - self.diff)
                    tmp = s_row[j]
                    s_row[j] = max(s_prev + self.getScore(seq1[i], seq2[j], mode), d_row[j], i_row[j], h_row[j])
                    s_prev = tmp

            for j in range(N - 1, -1, -1):
                I_ROW[j] = I_ROW[j + 1] - self.gapExtend
                H_ROW[j] = H
                S_ROW[j] = max(I_ROW[j], H_ROW[j])
                D_ROW[j] = S_ROW[j] - self.gapOpen

            for i in range(M - 1, imid - 1, -1):
                S_PREV = S_ROW[N]
            
                D_ROW[N] = D_ROW[N] - self.gapExtend
                H_ROW[N] = H
                S_ROW[N] = max(D_ROW[N], H_ROW[N])
                I_ROW[N] = S_ROW[N] - self.gapOpen
            
                for j in range(N - 1, -1, -1):
                    D_ROW[j] = max(D_ROW[j] - self.gapExtend, S_ROW[j] - self.gapOpen - self.gapExtend)
                    I_ROW[j] = max(I_ROW[j + 1] - self.gapExtend, S_ROW[j + 1] - self.gapOpen - self.gapExtend)
                    H_ROW[j] = max(H_ROW[j + 1], H_ROW[j], S_ROW[j + 1] - self.diff, S_ROW[j] - self.diff)
                    tmp = S_ROW[j]
                    S_ROW[j] = max(S_PREV + self.getScore(seq1[i + 1], seq2[j + 1], mode), D_ROW[j], I_ROW[j], H_ROW[j])
                    S_PREV = tmp
        
            hk = -math.inf
            df = -math.inf
            st = -math.inf
        
            jh = 0
            jd = 0
            js = 0
        
            # Find the point of intersection in which optimal alignment crosses to locate the partition point
            # of the second sequence.
            for j in range(N + 1):
                if h_row[j] + H_ROW[j] + self.diff > hk:
                    hk = h_row[j] + H_ROW[j] + self.diff
                    jh = j
                if d_row[j] + D_ROW[j] + self.gapOpen > df:
                    df = d_row[j] + D_ROW[j] + self.gapOpen
                    jd = j
                if s_row[j] + S_ROW[j] > st:
                    st = s_row[j] + S_ROW[j]
                    js = j

            if max(hk, df, st) == st:
                jmid = js
                final_score = st
                current = 'S'
            
            elif max(hk, df, st) == df:
                jmid = jd
                final_score = df
                current = 'D'
            
            else:
                jmid = jh 
                final_score = hk
                current = 'H'
            
            # Recursively call itself on the partitions of both sequences. 
            top_align = self.gap3_trace(seq1[1: imid + 1], seq2[1:jmid + 1], s, d, h, 
                                         S_ROW[jmid], D_ROW[jmid], H_ROW[jmid], curr, mode)
        
            bottom_align = self.gap3_trace(seq1[imid+1::], seq2[jmid+1::], s_row[jmid], 
                                            d_row[jmid], h_row[jmid], S, D, H, current, mode)
            
            # Merge the two generated optimal alignments together to solve the optimal alignment of the 
            # whole sequences.
            final_align_seq1 = top_align[1] + bottom_align[1]
            final_align_mid = top_align[2] + bottom_align[2]
            final_align_seq2 = top_align[3] + bottom_align[3]
        
            return [final_score, final_align_seq1, final_align_mid, final_align_seq2]
         
    # Method for gap3 alignment, which aligns both sequences end-to-end while finding multiple regions of 
    # similarities between the two sequences, and outputs the whole alignment. It is a generalized global 
    # alignment model defined to handle sequences with intermittent similarities. You can think of it as a
    # mix of global alignment and local alignment. 
    # @param seq1 : first sequence
    # @param seq2 : second sequence 
    # @param mode : determines the scoring system that the algorithm will use. Default is 0, which is the 
    #               scoring system for DNA/ RNA sequences. If not 0, the algorithm uses the scoring system 
    #               for protein sequences. 
    def gap3(self, seq1, seq2, mode=0):
        s = 0
        d = -self.gapOpen
        h = -self.diff
        if len(seq1) == 0 or len(seq2) == 0:
            print("One of the sequences is empty.")
            return

        arr = self.gap3_trace(seq1, seq2, s, d, h, s, d, h, 'S', mode)
        self.printAlignment(seq1, seq2, 0, 0, len(seq1), len(seq2), arr[0], arr[1], arr[2], arr[3])
    
    # Method for formatting the optimal alignment output
    # @param seq1 : first sequence
    # @param seq2 : second sequence 
    # @param start1 : the index of the character in the first sequence that starts the generated alignment
    # @param start2 : the index of the character in the second sequence that starts the generated alignment
    # @param end1 : the index of the last character in the first sequence that ends the generated alignment
    # @param end2 : the index of the last character in the second sequence that ends the generated alignment
    # @param score : the alignment score of the optimal alignment
    # @param align_seq1 : the alignment string consisting only the first sequence
    # @param align_mid : the alignment string consisting only the middle symbols that define the matches ('|'),
    #                    mismatches ('*'), difference blocks (" "), or same functionality amino acids ('='). 
    # @param align_seq2 : the alignment string consisting only the second sequence
    def printAlignment(self, seq1, seq2, start1, start2, end1, end2, score, align_seq1, align_mid, align_seq2):
        counter = 0
        curr1 = start1 + 1
        curr2 = start2 + 1
        max_space = len(str(max(len(seq1), len(seq2))))
        print("Alignment Score: " + str(score))
        print("Sequences Similarity: " + str(round((align_mid.count("|")/ len(align_mid)) * 100,2)) + "%\n")
        while counter <= len(align_seq1):
            print("\t" + "Sequence 1 > " + self.generateString('left', max_space, str(curr1)) + align_seq1[counter: counter + 79] + self.generateString('right', max_space, str(min(curr1 + 79 - align_seq1[counter: counter + 79].count('-') - 1, end1))))
            print("\t" + "             " + self.generateString('left', max_space, "") + align_mid[counter: counter + 79] + self.generateString('right', max_space, ""))
            print("\t" + "Sequence 2 > " + self.generateString('left', max_space, str(curr2)) + align_seq2[counter: counter + 79] + self.generateString('right', max_space, str(min(curr2 + 79 - align_seq2[counter: counter + 79].count('-') - 1, end2))))
            
            print("\n")
            curr1 = min(curr1 + 79 - align_seq1[counter: counter + 79].count('-'), end1)
            curr2 = min(curr2 + 79 - align_seq2[counter: counter + 79].count('-'), end2)
            counter += 79
    
    # Method for adding necessary number of spaces after and before numbers to make the output alignment
    # cleaner and clearer to see. 
    # @param position : indicates the position of the numbers in the alignment. Add spaces to the right of the
    #                   number if 'left', or left of the number if "right"
    # @param max_space : the maximum number of digits the number has, allows for precise padding.
    # @param string : the number in string format
    def generateString(self, position, max_space, string):
        if len(string) > max_space:
            return
        if position == 'left':
            return string + ((max_space - len(string)) * " ") + " "
        elif position == 'right':
            return " " + ((max_space - len(string)) * " ") + string
        else:
            return


In [72]:
p = Sequence_Alignment(mismatch=-10, gapOpen=40, gapExtend=2, diff_block=20)
A = "GCGCTCCGGGACGCCTTCCGCCGTCGGGAGCCCTACAACTACCTGCAGAGGGCCTATTACCAGGTGGGGAGCGGGCCGGGCAGTAGCCTTCCC"
A+="CAGAGCCCCCTAGCCGCAGGCACCAGAGGGTCCAAGACAAGACTGGAAGGGCACCTCGGGTTCGGGAGGAGCTGTGAGTGGCT"
B = "GGGAGCCTTACAACTACCTGCAGAGGGCCTACTACCAGGTGCGGGGGCCGGCCAGGGTGCTACCCCAAGCCTACTGACTGTCTTACTGG"
B+= "CAAGCTTCAGCGAGTCCAGGAGAAAGCTGGGAAGCCCCGCCGGGTCCGGGTCCGAGAGGAACTGTGAATGGCTGAGCCTGCTTCTCGAGGATCAGGC"
print("GAP3 Alignment: ")
p.gap3(A,B)
print("-----------------------------------------------------------------------------------------------------")
print("Global Alignment: ")
p.global_alignment(A,B)
print("-----------------------------------------------------------------------------------------------------")
print("Local Alignment: ")
p.local_alignment(A,B)

GAP3 Alignment: 
Alignment Score: 760.0
Sequences Similarity: 47.88%

	Sequence 1 > 1   GCGCTCCGGGACGCCTTCCGCCGTCGGGAGCCCTACAACTACCTGCAGAGGGCCTATTACCAGGTGGGGAGCGGGCCGG  79
	                 +++++++++++++++++++++++++|||||||*|||||||||||||||||||||||*|||||||||*||+++|||||||    
	Sequence 2 > 1                            GGGAGCCTTACAACTACCTGCAGAGGGCCTACTACCAGGTGCGG   GGGCCGG  79


	Sequence 1 > 80  GCAGTAGC    CTTCCCCAGAGCCCCCTAGCCGCA              GGCAC     CAGAGGGTCCAAGACAAGA 158
	                 *|||++++++++||*|||||+||+++||||++++++++++++++++++++||||++++++|||*|*|||||*||*||++    
	Sequence 2 > 80  CCAG    GGTGCTACCCCA AG   CCTA      CTGACTGTCTTACTGGCA AGCTTCAGCGAGTCCAGGAGAA   158


	Sequence 1 > 159 CT     GGAAGGGCACCT  CGGGTTCGG      GAGGAGCTGTGAGTGGCT                         176
	                 +++++++||||++||*||+++|||||*|||++++++|||||*||||||*|||||++++++++++++++++++++++++    
	Sequence 2 > 159   AGCTGGGAA  GCCCC GCCGGGTCCGGGTCCGAGAGGAACTGTGAATGGCTGAGCCTGCTTCTCGAGGATCAGGC 186


---------

In [73]:
p = Sequence_Alignment(diff_block=60)
A = "GCGCTCCGGGACGCCTTCCGCCGTCGGGAGCCCTACAACTACCTGCAGAGGGCCTATTACCAGGTGGGGAGCGGGCCGGGCAGTAGCCTTCCC"
A+="CAGAGCCCCCTAGCCGCAGGCACCAGAGGGTCCAAGACAAGACTGGAAGGGCACCTCGGGTTCGGGAGGAGCTGTGAGTGGCT"
B = "GGGAGCCTTACAACTACCTGCAGAGGGCCTACTACCAGGTGCGGGGGCCGGCCAGGGTGCTACCCCAAGCCTACTGACTGTCTTACTGG"
B+= "CAAGCTTCAGCGAGTCCAGGAGAAAGCTGGGAAGCCCCGCCGGGTCCGGGTCCGAGAGGAACTGTGAATGGCTGAGCCTGCTTCTCGAGGATCAGGC"
p.gap3(A,B)

Alignment Score: 354.0
Sequences Similarity: 26.43%

	Sequence 1 > 1   GCGCTCCGGGACGCCTTCCGCCGTCGGGAGCCCTACAACTACCTGCAGAGGGCCTATTACCAGGTGGGGAGCGGGCCGG  79
	                 +++++++++++++++++++++++++|||||||*|||||||||||||||||||||||*|||||||||*||   |||||||    
	Sequence 2 > 1                            GGGAGCCTTACAACTACCTGCAGAGGGCCTACTACCAGGTGCGG---GGGCCGG  76


	Sequence 1 > 80  GCAGTAGCCTTCCCCAGAGCCCCCTAGCCGCAGGCACCAGAGGGTCCAAGACAAGACTGGAAGGGCACCT          158
	                 *|||+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
	Sequence 2 > 77  CCAG                                                                  GGTGCTACC 155


	Sequence 1 > 159                                                                        CGGGTTCG 176
	                 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++|||||*||    
	Sequence 2 > 156 CCAAGCCTACTGACTGTCTTACTGGCAAGCTTCAGCGAGTCCAGGAGAAAGCTGGGAAGCCCCGCCGGGTCCGGGTCCG 186


	Sequence 1 > 176 GGAGG

In [19]:
p = Sequence_Alignment(gapExtend=2)
p.alignInput("gene.fna", "mutant.fna", "gap3")

Aligning NW_021636346.1:c40256-27400 with NW_021636346.1:c40256-27400 using gap3 alignment: 

Alignment Score: 22500.0
Sequences Similarity: 33.33%

	Sequence 1 > 1    TCACGGAGCTTAAACAAGGCCCTCTATAGCCTCTGTCTGCTTACTGTGCCACAAAAACGCCCACCTTCATAACGGCTTC   79
	                  |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||     
	Sequence 2 > 1    TCACGGAGCTTAAACAAGGCCCTCTATAGCCTCTGTCTGCTTACTGTGCCACAAAAACGCCCACCTTCATAACGGCTTC   79


	Sequence 1 > 80   TTGCATAAGCGACTGTGGAAGAAAGCTGGGGAGAGTGGTGGAGTTGATTGCCATGGGGTTTTCGACCCAGAAGCCCACC  158
	                  |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||     
	Sequence 2 > 80   TTGCATAAGCGACTGTGGAAGAAAGCTGGGGAGAGTGGTGGAGTTGATTGCCATGGGGTTTTCGACCCAGAAGCCCACC  158


	Sequence 1 > 159  ACCACCACTTTGCGGCCCTTTTGCCCCTCCTCTTTCTGGACTTCGCGCTCTTGTCGTTCCTTTCGCGTCGCTTCTTCTT  237
	                  |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||     
	Sequence 2 > 1

In [57]:
p = Sequence_Alignment()
p.manualAlign('protein')

Enter a PROTEIN sequence to be aligned: gQQTA
------------------------------------------------------------------------------------------------
Enter another PROTEIN sequence to be aligned: ooa
***PROTEINsequences should contain only these characters: AGVLISTCMPFYWDENQKRH, and no spaces are allowed ! ***

Enter another PROTEIN sequence to be aligned: AQTYHC
------------------------------------------------------------------------------------------------
Please choose an alignment mode: 
	1. Type 'global' for global alignment
	2. Type 'local' for local alignment
	3. Type 'gap3' for gap3 alignment

Enter your desired alignment type: global
------------------------------------------------------------------------------------------------

Outputting global Alignment ...

Alignment Score: -40.0
Sequences Similarity: 16.67%

	Sequence 1 > 1 gQ-QTA 5
	               *| ***  
	Sequence 2 > 1 AQTYHC 6




In [74]:
p = Sequence_Alignment(mismatch=-10, gapOpen=40, gapExtend=2, diff_block=20)
A = "RFQLVEQLHWCGYNGEVTGCGQSQKMHRKPFNRRQVYNKAHMISLGVQPLQFQYRFTINYKSKWIVCNFDVPFWVENELPRADTRWAYSMIQKDVGHQAAV"
B = "FTQQDEFVWLEWNDLRVKQEIEVDSAYEPMIWKECKANSYLNWQWMRGIEDHVADFVIFHHQNWQFVMIVIQMWAEMFEDDSICATARCWIYNRFLWSHRP"

print("GAP3 Alignment: ")
p.gap3(A,B)
print("-----------------------------------------------------------------------------------------------------")
print("Global Alignment: ")
p.global_alignment(A,B)
print("-----------------------------------------------------------------------------------------------------")
print("Local Alignment: ")
p.local_alignment(A,B)

GAP3 Alignment: 
Alignment Score: -20.0
Sequences Similarity: 0.0%

	Sequence 1 > 1   RFQLVEQLHWCGYNGEVTGCGQSQKMHRKPFNRRQVYNKAHMISLGVQPL                               79
	                 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
	Sequence 2 > 1                                                     FTQQDEFVWLEWNDLRVKQEIEVDSAYEP  79


	Sequence 1 > 80                                     QFQYRFTINYKSKWIVCNFDVPFWVENELPRADTRWAYSMIQKD 101
	                 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
	Sequence 2 > 80  MIWKECKANSYLNWQWMRGIEDHVADFVIFHHQNW                                             101


	Sequence 1 > 101 VGHQAAV                                      101
	                 ++++++++++++++++++++++++++++++++++++++++++++    
	Sequence 2 > 101        QFVMIVIQMWAEMFEDDSICATARCWIYNRFLWSHRP 101


-----------------------------------------------------------------------------------------------------
Global Alig